In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# ============================================================================
# CONFIGURATION
# ============================================================================
FILENAME = "raw_binary_data/raw_data_ch1_20251006_143204.bin"  # Change this to your file
SAMPLING_RATE = 30000  # Hz
DURATION_TO_PLOT = 1.0  # seconds

# ============================================================================
# Read Raw Data
# ============================================================================

if not os.path.exists(FILENAME):
    print(f"Error: File '{FILENAME}' not found")
else:
    # Get file information
    file_size = os.path.getsize(FILENAME)
    expected_samples = file_size // 8  # 8 bytes per double
    
    print(f"File: {FILENAME}")
    print(f"Size: {file_size:,} bytes")
    print(f"Expected samples: {expected_samples:,}")
    print(f"Expected duration: {expected_samples / SAMPLING_RATE:.2f} seconds")
    print()
    
    # Read as flat array of doubles
    data = np.fromfile(FILENAME, dtype=np.float64)
    print(f"Successfully read {len(data):,} samples")
    print(f"Data type: {data.dtype}")
    print(f"Shape: {data.shape}")
    print()

# ============================================================================
# Data Statistics
# ============================================================================

print("Data Statistics:")
print(f"  Mean: {np.mean(data):.2f} µV")
print(f"  Std Dev: {np.std(data):.2f} µV")
print(f"  Min: {np.min(data):.2f} µV")
print(f"  Max: {np.max(data):.2f} µV")
print()

# Check for issues
if np.any(np.isnan(data)):
    print("WARNING: Data contains NaN values!")
if np.any(np.isinf(data)):
    print("WARNING: Data contains Inf values!")

# Show first few samples
print("First 20 samples (µV):")
print(data[:20])

# ============================================================================
# Plot Data
# ============================================================================

samples_to_plot = int(DURATION_TO_PLOT * SAMPLING_RATE)
samples_to_plot = min(samples_to_plot, len(data))

time = np.arange(samples_to_plot) / SAMPLING_RATE

plt.figure(figsize=(14, 6))
plt.plot(time, data[:samples_to_plot], 'b-', linewidth=0.5)
plt.xlabel('Time (seconds)', fontsize=12)
plt.ylabel('Amplitude (µV)', fontsize=12)
plt.title(f'Neural Signal - First {DURATION_TO_PLOT} seconds', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================================
# Plot Power Spectral Density
# ============================================================================

from scipy import signal

# Use first 10 seconds of data for PSD
psd_duration = min(10.0, len(data) / SAMPLING_RATE)
psd_samples = int(psd_duration * SAMPLING_RATE)

freqs, psd = signal.welch(data[:psd_samples], 
                          fs=SAMPLING_RATE, 
                          nperseg=4096)

plt.figure(figsize=(14, 6))
plt.semilogy(freqs, psd)
plt.xlabel('Frequency (Hz)', fontsize=12)
plt.ylabel('Power Spectral Density (µV²/Hz)', fontsize=12)
plt.title('Power Spectral Density', fontsize=14)
plt.xlim([0, 500])  # Focus on 0-500 Hz range
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ============================================================================
# Optional: Save as CSV (with downsampling for large files)
# ============================================================================

# Uncomment to save as CSV:
# DOWNSAMPLE_FACTOR = 10  # Reduce file size by this factor
# data_downsampled = data[::DOWNSAMPLE_FACTOR]
# time_downsampled = np.arange(len(data_downsampled)) * DOWNSAMPLE_FACTOR / SAMPLING_RATE
# 
# df = pd.DataFrame({
#     'time_sec': time_downsampled,
#     'amplitude_uV': data_downsampled
# })
# 
# output_filename = FILENAME.replace('.bin', '_downsampled.csv')
# df.to_csv(output_filename, index=False)
# print(f"Saved to: {output_filename}")
# print(f"New sampling rate: {SAMPLING_RATE/DOWNSAMPLE_FACTOR:.0f} Hz")